In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
import time
import pickle

import numpy as np
import pandas as pd

import rdkit
import rdkit.Chem as chem
import rdkit.Chem.AllChem as allchem

from gryffin import Gryffin

from MolarInterface import MolarInterface

2022-08-16 13:47:33.049042: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda/lib64
2022-08-16 13:47:33.049059: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Make connection to the MolarDB

In [5]:
db = MolarInterface(
    db_name="madness_laser",
    fragments=("fragment_a", "fragment_b", "fragment_c")
)

## Retrieve the first set of molecules in the DB

We use a filtered query to return all of the molecules for which we have tried to measure the optical properties. 

In [6]:
previous_syntheses = db.get_all_syntheses()
print(previous_syntheses.shape)
previous_syntheses.head()

(188, 12)


,product.hid,product.smiles,product.optical_properties,fragment_a.hid,fragment_a.smiles,fragment_b.hid,fragment_b.smiles,fragment_c.hid,fragment_c.smiles,synthesis.synthesis_id,synthesis.status,synthesis.molecule_id
0,A001B003C001,C(=Cc1ccc(-n2c3ccccc3c3ccccc32)cc1)c1ccc(-c2cc...,"{'PL_lambda_max': 443.6, 'validation_status': ...",A001,CC1(C)OB(c2ccc(-n3c4ccccc4c4ccccc43)cc2)OC1(C)C,B003,C[N+]12CC(=O)O[B-]1(/C=C/Br)OC(=O)C2,C001,Brc1ccc(-c2ccc(Br)cc2)cc1,695c1c55-bf6a-4265-b68d-4655999a7b85,DONE,539c5e5e-25fa-457d-a2c0-ecf3f88c36f3
1,A001B003C002,C(=Cc1ccc(-n2c3ccccc3c3ccccc32)cc1)c1ccc(C=Cc2...,"{'PL_lambda_max': 448.2, 'validation_status': ...",A001,CC1(C)OB(c2ccc(-n3c4ccccc4c4ccccc43)cc2)OC1(C)C,B003,C[N+]12CC(=O)O[B-]1(/C=C/Br)OC(=O)C2,C002,Ic1ccc(I)cc1,9a71f775-c341-42b1-ab7a-cd0917b5e80f,DONE,3275cec5-919c-4f18-bb4c-2ec941f7db3b
2,A001B003C003,Cc1cc(C=Cc2ccc(-n3c4ccccc4c4ccccc43)cc2)c(C)cc...,"{'PL_lambda_max': 453.6, 'validation_status': ...",A001,CC1(C)OB(c2ccc(-n3c4ccccc4c4ccccc43)cc2)OC1(C)C,B003,C[N+]12CC(=O)O[B-]1(/C=C/Br)OC(=O)C2,C003,Cc1cc(Br)c(C)cc1Br,8cf70e33-a879-4ab7-879f-5284da51c300,DONE,63bd1d6d-3e81-48ae-9535-e04dc6d9d32f
3,A001B003C004,Fc1c(F)c(-c2c(F)c(F)c(C=Cc3ccc(-n4c5ccccc5c5cc...,"{'PL_lambda_max': 541.1, 'validation_status': ...",A001,CC1(C)OB(c2ccc(-n3c4ccccc4c4ccccc43)cc2)OC1(C)C,B003,C[N+]12CC(=O)O[B-]1(/C=C/Br)OC(=O)C2,C004,Fc1c(F)c(-c2c(F)c(F)c(Br)c(F)c2F)c(F)c(F)c1Br,512b328b-6a19-4f7d-9cc1-a136edc9f344,DONE,a5e39076-8b93-40f8-93b0-8b5edc65ef53
4,A001B003C005,CC1(C)c2cc(C=Cc3ccc(-n4c5ccccc5c5ccccc54)cc3)c...,"{'PL_lambda_max': 445.1, 'validation_status': ...",A001,CC1(C)OB(c2ccc(-n3c4ccccc4c4ccccc43)cc2)OC1(C)C,B003,C[N+]12CC(=O)O[B-]1(/C=C/Br)OC(=O)C2,C005,CC1(C)c2cc(Br)ccc2-c2ccc(Br)cc21,7fbd4503-b19e-4163-9105-ee34b38e4a3e,DONE,9131832d-c235-4a6a-9183-ed12f0f8ac3e


In [12]:
# get the sub dataframe of all molecules that are either SHIPPED OR IN PROCESSING
# these will be constrained in the Gryffin acquisition
df_proc = previous_syntheses[previous_syntheses['synthesis.status'].isin(["AVAILABLE", "ACQUIRED", "PROCESSING", "SYNTHESIZED", "SHIPPED", "RECEIVED"])]
print('NUM IN PROGRESS : ', df_proc.shape[0])

# get all the entries for which the status is DONE
df_done = previous_syntheses[previous_syntheses['synthesis.status'].isin(['DONE', 'FAILED'])]
# ATTN: Hackily adding column for synthesis type
df_done['synthesis_generation'] = 'gen_1'
print('NUM FINISHED : ', df_done.shape[0])

# print(f'PERCENT DONE {(df_done.shape[0] / (33*30*165) *100):.3f}')
print(f'PERCENT DONE {(df_done.shape[0] / (42*68*161) *100):.3f}')  # TODO: Make space calculaiton automatic from DB?

df_proc

NUM IN PROGRESS :  0
NUM FINISHED :  188
PERCENT DONE 0.041


,product.hid,product.smiles,product.optical_properties,fragment_a.hid,fragment_a.smiles,fragment_b.hid,fragment_b.smiles,fragment_c.hid,fragment_c.smiles,synthesis.synthesis_id,synthesis.status,synthesis.molecule_id


In [26]:
# get the fragment details for the fragments

from logging import critical


def make_gryffin_observations(df_done):
    """
    prepare typical list of dictionary-style observations for Gryffin. This function
    considers all failed experiments as nan-valued objectives. We do not consider the cause
    for failure here (future improvement). This function is quite slow with all the calls to 
    the db about the fragments - make more efficient
    """
    # prepare observation list for Gryffin
    observations = []

    for _, row in df_done.iterrows():
        row = row.to_dict()

        if row["synthesis.status"]=='DONE':
            try:
                gain_cross_section = row["product.optical_properties"]["gain_cross_section"]
            except KeyError:
                print(f"Target property could not be found for {row['hid']}")

        elif row['synthesis.status']=='FAILED':
            # failed experiment, pass value to gryffin with nan objective value
            gain_cross_section = np.nan

        else:
            raise NotImplementedError

        observations.append(
            {
                'fragment_a': row["fragment_a.hid"],
                'fragment_b': row["fragment_b.hid"],
                'fragment_c': row["fragment_c.hid"],
                'synthesis_gen': row["synthesis_generation"],
                'obj': gain_cross_section
            }
        )
        
    return observations

def known_constraints(param):
    '''
    known constraint that the suggested molecule must not be either shipping or
    in progress. Avoids redundant suggestions
    '''
    # check if the param is within the known molecules
    query_hid = ''.join([param['fragment_a'], param['fragment_b'], param['fragment_c']])
    crit1 = query_hid not in proc_hids
    
    # check to see if the sample can be made in at least one single location
    crit2 = False
    # TODO: this in only for the first batch, constrain to Toronto ONLY!
    for lab_frags in [toronto_frags, uiuc_frags]:
        if (
            (param['fragment_a'] in lab_frags['fragment_a']) & (param['fragment_b'] in lab_frags['fragment_b']) & (param['fragment_c'] in lab_frags['fragment_c'])):
            crit2=True

    # Make sure generation is gen_2
    crit_3 = (param['synthesis_gen'] == 'gen_2')  # ugly, but it works for now
            
    return crit1 & crit2 & crit_3


In [27]:

# get hids of molecules in process
proc_hids = df_proc["product.hid"].tolist()
proc_hids

[]

In [28]:
# get gryffin observations
observations = make_gryffin_observations(df_done)

observations

[{'fragment_a': 'A001',
  'fragment_b': 'B003',
  'fragment_c': 'C001',
  'synthesis_gen': 'gen_1',
  'obj': 1.869e-16},
 {'fragment_a': 'A001',
  'fragment_b': 'B003',
  'fragment_c': 'C002',
  'synthesis_gen': 'gen_1',
  'obj': 1.968e-16},
 {'fragment_a': 'A001',
  'fragment_b': 'B003',
  'fragment_c': 'C003',
  'synthesis_gen': 'gen_1',
  'obj': 2.309e-16},
 {'fragment_a': 'A001',
  'fragment_b': 'B003',
  'fragment_c': 'C004',
  'synthesis_gen': 'gen_1',
  'obj': 6.72e-17},
 {'fragment_a': 'A001',
  'fragment_b': 'B003',
  'fragment_c': 'C005',
  'synthesis_gen': 'gen_1',
  'obj': 2.6859999999999997e-16},
 {'fragment_a': 'A001',
  'fragment_b': 'B003',
  'fragment_c': 'C006',
  'synthesis_gen': 'gen_1',
  'obj': 1.19e-17},
 {'fragment_a': 'A001',
  'fragment_b': 'B003',
  'fragment_c': 'C008',
  'synthesis_gen': 'gen_1',
  'obj': 1.89e-17},
 {'fragment_a': 'A001',
  'fragment_b': 'B003',
  'fragment_c': 'C011',
  'synthesis_gen': 'gen_1',
  'obj': 2.442e-16},
 {'fragment_a': 'A001'

In [29]:
# get all the unique frags in the previous observations to add onto the available 
# fragments (otherwise Gryffin will crash)
a_prev_hids = list(set([o['fragment_a'] for o in observations]))
b_prev_hids = list(set([o['fragment_b'] for o in observations]))
c_prev_hids = list(set([o['fragment_c'] for o in observations]))

In [30]:
def get_lab_frags(lab):
    ''' returns the set of available fragments for a specific laboratory
    '''
    a_hids = db.get_available_fragments("fragment_a", lab)["molecule.hid"].tolist()
    b_hids = db.get_available_fragments("fragment_b", lab)["molecule.hid"].tolist()
    c_hids = db.get_available_fragments("fragment_c", lab)["molecule.hid"].tolist()
    
    return {'fragment_a': a_hids, 'fragment_b': b_hids, 'fragment_c': c_hids}

def get_frags():
    ''' return all the unique molecules across all labs
    '''
    a_hids_all = []
    b_hids_all = []
    c_hids_all = []
    
    for lab in ['Toronto', 'Illinois']:
        lab_frags = get_lab_frags(lab)
        a_hids_all.extend(lab_frags['fragment_a'])
        b_hids_all.extend(lab_frags['fragment_b'])
        c_hids_all.extend(lab_frags['fragment_c'])
    
    a_hids_all = list(set(a_hids_all))
    b_hids_all = list(set(b_hids_all))
    c_hids_all = list(set(c_hids_all))
    
    return a_hids_all, b_hids_all, c_hids_all
    

In [31]:
a_hids_all, b_hids_all, c_hids_all = get_frags()

# add fragments from previous iterations
a_hids_all = list(set(a_hids_all + a_prev_hids))
b_hids_all = list(set(b_hids_all + b_prev_hids))  # ATTN: Incorrect because HIDs overlap fragments for MIDA/TIDA
c_hids_all = list(set(c_hids_all + c_prev_hids))

print('NUM A FRAGS : ', len(a_hids_all))
print('NUM B FRAGS : ', len(b_hids_all))
print('NUM C FRAGS : ', len(c_hids_all))

print('NUM ACCESSIBLE MOLS : ', len(a_hids_all)*len(b_hids_all)*len(c_hids_all))  # This doesn't actually matter

NUM A FRAGS :  33
NUM B FRAGS :  55
NUM C FRAGS :  161
NUM ACCESSIBLE MOLS :  292215


In [32]:
# get toronto and UIUC available frags
toronto_frags = get_lab_frags('Toronto')
uiuc_frags = get_lab_frags('Illinois')

## Build Gryffin 

In [33]:
def get_descriptors(hid, descriptors):
    ''' looks up and returns descriptors for a particular fragment
    
    Args:
        hid (str): human readable identifier of the fragment
        descriptors (pd.DataFrame): dataframe corresponding to the type of fragment
    '''
    print(hid)
    return descriptors[descriptors.hid==hid].iloc[0, 1:].values.astype(np.float).tolist()

def mol_hid_from_params(params):
    mol_hids = []
    for param in params:
        mol_hids.append(''.join([param['fragment_a'], param['fragment_b'], param['fragment_c']]))
    
    return mol_hids
        

In [34]:
# load descriptors from disk
pca_desc_a = pd.read_csv('descriptors/gen2_pca_desc_a.csv', index_col=None)
pca_desc_b = pd.read_csv('descriptors/gen2_pca_desc_b.csv', index_col=None)
pca_desc_c = pd.read_csv('descriptors/gen2_pca_desc_c.csv', index_col=None)
print('NUM DESC A ', pca_desc_a.shape[1]-1)
print('NUM DESC B ', pca_desc_b.shape[1]-1)
print('NUM DESC C ', pca_desc_c.shape[1]-1)

NUM DESC A  30
NUM DESC B  30
NUM DESC C  35


In [35]:
# set the available fragments to make the Gryffin search space 
# TODO: change this to only those available
avail_frags_a = a_hids_all
avail_frags_b = b_hids_all
avail_frags_c = c_hids_all
print('NUM AVAIL FRAGS A : ', len(avail_frags_a))
print('NUM AVAIL FRAGS B : ', len(avail_frags_b))
print('NUM AVAIL FRAGS C : ', len(avail_frags_c))

NUM AVAIL FRAGS A :  33
NUM AVAIL FRAGS B :  55
NUM AVAIL FRAGS C :  161


In [36]:
# Gryffin config

#sampling_strategies = np.linspace(-1, 1, 40)
sampling_strategies_1 = np.linspace(0.6, 1, 30)
sampling_strategies_2 = np.linspace(-1, 0.5, 10)
sampling_strategies = np.concatenate((sampling_strategies_1, sampling_strategies_2))
#sampling_strategies = np.linspace(-1, 0.45, 20)

# generate descriptors
FRAG_A_DESC = {i: get_descriptors(i, pca_desc_a) for i in avail_frags_a}
FRAG_B_DESC = {i: get_descriptors(i, pca_desc_b) for i in avail_frags_b}
FRAG_C_DESC = {i: get_descriptors(i, pca_desc_c) for i in avail_frags_c}
SYNTH_GEN = {i: None for i in ["gen_1", "gen_2"]}

# FRAG_A_DESC = {i: None for i in avail_frags_a}
# FRAG_B_DESC = {i: None for i in avail_frags_b}
# FRAG_C_DESC = {i: None for i in avail_frags_c}

# fwa = feasibility-weighted acquisition
# fia = feasibility-interpolated acquisition
# fca = feasibility-constrained acquisition --> use, param=0.2 ish works best on cat benchmarks

config = {
     "general": {
             "backend": 'tensorflow',
             "num_cpus": 4,
             "auto_desc_gen": False, # dynamic Gryffin
             "batches": 1,
             "sampling_strategies": 1,
             "feas_approach": 'fca',
             "feas_param": 0.2,
             "boosted":  True,
             "caching": False,
             "random_seed": 22031996,
             "acquisition_optimizer": 'genetic',
             "verbosity": 3
                },
    "parameters": [
        {"name": "fragment_a", "type": "categorical", "category_details": FRAG_A_DESC },
        {"name": "fragment_b", "type": "categorical", "category_details": FRAG_B_DESC },
        {"name": "fragment_c", "type": "categorical", "category_details": FRAG_C_DESC },
        {"name": "synthesis_gen", "type": "categorical", "category_details": SYNTH_GEN},
      
    ],
    "objectives": [
        {"name": "obj", "goal": "max"},
    ]
}

A032
A029
A015
A016
A028
A034
A012
A040
A021
A003
A014
A002
A027
A041
A023
A020
A042
A037
A038
A022
A017
A010
A018
A011
A024
A026
A035
A001
A039
A007
A008
A036
A004
B057
B021
B038
B040
B015
B017
B032
B008
B022
B051
B052
B010
B011
B043
B056
B045
B025
B012
B016
B026
B003
B009
B036
B031
B046
B060
B049
B033
B055
B001
B002
B024
B062
B054
B023
B020
B044
B019
B018
B035
B013
B006
B042
B014
B039
B004
B048
B005
B047
B063
B007
B068
B037
B059
B058
C161
C099
C017
C075
C048
C057
C055
C032
C133
C098
C001
C144
C157
C052
C125
C116
C132
C018
C062
C016
C118
C008
C105
C160
C137
C044
C152
C106
C068
C064
C043
C025
C072
C074
C085
C071
C115
C027
C066
C011
C159
C059
C046
C076
C110
C104
C129
C135
C049
C127
C094
C126
C156
C014
C020
C111
C101
C093
C010
C009
C002
C038
C090
C096
C083
C130
C122
C006
C028
C047
C037
C107
C015
C097
C079
C113
C149
C077
C139
C026
C150
C069
C134
C067
C082
C151
C024
C109
C155
C019
C086
C012
C142
C147
C123
C033
C138
C141
C095
C034
C131
C056
C091
C061
C102
C153
C145
C143
C078
C060
C004
C051


In [37]:
# instantiate gryffin
gryffin = Gryffin(config_dict=config, known_constraints=known_constraints)

In [60]:
# # recommend a batch of samples
# samples = []
# for ix, sampling_strat in enumerate(sampling_strategies):
    
#     sample = gryffin.recommend(observations, sampling_strategies=[sampling_strat])
#     print(f'\nSAMPLING STRATEGY : {sampling_strat} SAMPLE : {sample}\n')
#     samples.extend(sample) 

samples = gryffin.recommend(observations, sampling_strategies=sampling_strategies)
    

188 observations found

2022-08-16 18:17:39.937243: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-16 18:17:39.992237: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-16 18:17:39.994628: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda/lib64
2022-08-16 18:17:39.994967: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11:

Bayesian neural network trained in 54.6 s

1600 random proposals drawn in 37.4 s

1600 proposals optimized in 10.7 s using 4 CPUs

...

1600 proposals optimized in 10.9 s using 4 CPUs

Acquisition tasks for 40 sampling strategies performed in 7 min 57 s

40 samples selected in 1 min 19 s

Memory usage: 5 GB, 5054 MB, 700 kB

Overall time required: 10 min 17 s

In [61]:
def check_duplicates(samples, observations):
    ''' returns True if there are duplicates, False if there are none
    '''
    sample_hids = [''.join([s['fragment_a'],s['fragment_b'],s['fragment_c']]) for s in samples]
    observ_hids = [''.join([o['fragment_a'],o['fragment_b'],o['fragment_c']]) for o in observations]
    
    # check duplicates over previous observations 
    is_obs_dup = any(s in observ_hids for s in sample_hids)
    
    # check duplicates within the batch
    is_samp_dup = len(set(sample_hids))<len(sample_hids)

    # for sample in set(sample_hids):
    #     print(sample, sample_hids.count(sample))
    
    return is_samp_dup, is_obs_dup

check_duplicates(samples, observations)

(False, False)

In [62]:
samples

[{'fragment_a': 'A014',
  'fragment_b': 'B033',
  'fragment_c': 'C009',
  'synthesis_gen': 'gen_2'},
 {'fragment_a': 'A014',
  'fragment_b': 'B052',
  'fragment_c': 'C009',
  'synthesis_gen': 'gen_2'},
 {'fragment_a': 'A024',
  'fragment_b': 'B033',
  'fragment_c': 'C009',
  'synthesis_gen': 'gen_2'},
 {'fragment_a': 'A022',
  'fragment_b': 'B033',
  'fragment_c': 'C009',
  'synthesis_gen': 'gen_2'},
 {'fragment_a': 'A028',
  'fragment_b': 'B033',
  'fragment_c': 'C009',
  'synthesis_gen': 'gen_2'},
 {'fragment_a': 'A039',
  'fragment_b': 'B033',
  'fragment_c': 'C009',
  'synthesis_gen': 'gen_2'},
 {'fragment_a': 'A027',
  'fragment_b': 'B033',
  'fragment_c': 'C009',
  'synthesis_gen': 'gen_2'},
 {'fragment_a': 'A026',
  'fragment_b': 'B033',
  'fragment_c': 'C009',
  'synthesis_gen': 'gen_2'},
 {'fragment_a': 'A023',
  'fragment_b': 'B033',
  'fragment_c': 'C009',
  'synthesis_gen': 'gen_2'},
 {'fragment_a': 'A014',
  'fragment_b': 'B059',
  'fragment_c': 'C009',
  'synthesis_gen': 

In [63]:
rxn_transmut = allchem.ReactionFromSmarts("[*:1][Br,I]>>[*:1][Po]")
rxn_b_c = allchem.ReactionFromSmarts(
    "[Br,I][*:1].C[N+]12CC(=O)O[B-]1([*:2])OC(=O)C2>>[*:1]-[*:2]"
)
rxn_a_b_c = allchem.ReactionFromSmarts("CC1(C)OB([*:1])OC1(C)C.[*:2][Po]>>[*:1]-[*:2]")
rxn_a_b_c2 = allchem.ReactionFromSmarts("[*:1]B(O)O.[*:2][Po]>>[*:1]-[*:2]")


def assemble_fragments(hid_a, hid_b, hid_c):
    smiles_a = db.get_molecule(hid_a, identifier_type='hid').at[0, 'smiles']
    smiles_b = db.get_molecule(hid_b, identifier_type='hid').at[0, 'smiles']
    smiles_c = db.get_molecule(hid_c, identifier_type='hid').at[0, 'smiles']

    a = chem.MolFromSmiles(smiles_a)
    b = chem.MolFromSmiles(smiles_b)
    c = chem.MolFromSmiles(smiles_c)
    if a is None or b is None or c is None:
        raise ValueError("Invalid SMILES")

    # Building the molecule... there is probably a better way
    b_transmut = rxn_transmut.RunReactants([b])[0][0]
    b_c = rxn_b_c.RunReactants([c, b_transmut])[0][0]
    b_c_b = rxn_b_c.RunReactants([b_c, b_transmut])[0][0]
    try:
        a_b_c_b = rxn_a_b_c.RunReactants([a, b_c_b])[0][0]
        a_b_c_b_a = rxn_a_b_c.RunReactants([a, a_b_c_b])[0][0]
    except:
        a_b_c_b = rxn_a_b_c2.RunReactants([a, b_c_b])[0][0]
        a_b_c_b_a = rxn_a_b_c2.RunReactants([a, a_b_c_b])[0][0]
    return chem.MolToSmiles(a_b_c_b_a)

In [64]:
molecule_smiles = []
molecule_hids = []

for sample in samples:
    molecule_hids.append(''.join([sample['fragment_a'],sample['fragment_b'],sample['fragment_c']]))
    molecule_smiles.append(assemble_fragments(sample['fragment_a'], sample['fragment_b'], sample['fragment_c']))
    

In [65]:
molecule_mols = [chem.MolFromSmiles(s) for s in molecule_smiles]

img = chem.Draw.MolsToGridImage(
    molecule_mols, molsPerRow=5, subImgSize=(400,400), returnPNG=False, legends=molecule_hids,
)
img.save('round_3_samples_all.png')

In [66]:
with open('round_3_samples_all.txt', 'w') as f:
    for smile, hid in zip(molecule_smiles, molecule_hids):
        f.write(f'{smile}\t{hid}\n')


## Write the new target molecules/syntheses to the DB

In [67]:
for sample, smi in zip(samples, molecule_smiles):
    print(f'>> Creating DB entry for sample {sample["fragment_a"]} {sample["fragment_b"]} {sample["fragment_c"]}...')
    db.create_target_compound(
        fragments=[sample["fragment_a"], sample["fragment_b"], sample["fragment_c"]],
        smiles=smi
    )

>> Creating DB entry for sample A014 B033 C009...
>> Creating DB entry for sample A014 B052 C009...
>> Creating DB entry for sample A024 B033 C009...
>> Creating DB entry for sample A022 B033 C009...
>> Creating DB entry for sample A028 B033 C009...
>> Creating DB entry for sample A039 B033 C009...
>> Creating DB entry for sample A027 B033 C009...
>> Creating DB entry for sample A026 B033 C009...
>> Creating DB entry for sample A023 B033 C009...
>> Creating DB entry for sample A014 B059 C009...
>> Creating DB entry for sample A014 B051 C009...
>> Creating DB entry for sample A014 B057 C009...
>> Creating DB entry for sample A021 B033 C009...
>> Creating DB entry for sample A020 B033 C009...
>> Creating DB entry for sample A042 B033 C009...
>> Creating DB entry for sample A018 B033 C009...
>> Creating DB entry for sample A003 B049 C087...
>> Creating DB entry for sample A003 B039 C151...
>> Creating DB entry for sample A003 B039 C067...
>> Creating DB entry for sample A003 B048 C151...
